# Bullet Forge Model Fine Tuning


## Step 1: Instantiate Global Parameters

In [1]:
# Max length of tokens a user may enter for summarization
# Increasing this beyond 512 may increase compute time significantly
max_input_token_length = 512
# Max length of tokens the model should output for the summary
# Approximately the number of tokens it may take to generate a bullet
max_output_token_length = 35
# Beams to use for beam search algorithm
# Increased beams means increased quality, but increased compute time
number_of_beams = 5

## Step 2: Fine Tune Checkpoint Model

In [4]:
# Fine Tuning Scripts
import os
import sys
import signal
import re
from loguru import logger
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import os
from rouge_score import rouge_scorer
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    get_linear_schedule_with_warmup,
)

from scripts.files import load_jsonl_data
from scripts.bullet_patterns import *
from scripts.rich_logger import *


input_model = input(
    "What is the target model's relative directory path or checkpoint name on Hugging Face?"
)

# Model fine tuning parameter control object
model_params = {
    # Name of the pre-trained model or checkpoint name that will be fine-tuned
    "MODEL": f"{input_model}",
    # Number of examples per batch during training
    # Larger batch sizes require more memory, but can speed up training
    "TRAIN_BATCH_SIZE": 32,
    # Number of examples per batch during validation
    # Larger batch sizes require more memory, but can speed up the validation process
    "VALID_BATCH_SIZE": 32,
    # Number of full passes through the entire training dataset
    # More epochs can lead to better performance, but risk over-fitting
    "TRAIN_EPOCHS": 16,
    # Number of full passes through the entire validation dataset
    # Typically kept to a single epoch as the validation set does not need to be repeatedly passed
    "VAL_EPOCHS": 1,
    # Determines the step size during gradient descent
    # Affects how quickly or slowly a model learns
    # Too high can cause instability, too low can cause slow learning
    "LEARNING_RATE": 1e-3,
    # Maximum number of tokens from source text that model accepts
    # Longer sequences will be truncated to this length
    "MAX_SOURCE_TEXT_LENGTH": max_input_token_length,
    # Maximum number of tokens from target text that model generates
    # Longer sequences will be truncated to this length
    "MAX_TARGET_TEXT_LENGTH": max_output_token_length,
    # Random seed to ensure reproducibility
    # Using the same seed will yield the same model given the same data and training process
    "SEED": 88,
    # Number of alternative sequences generated at each step
    # More beams improve results, but increase computation
    "NUM_BEAMS": number_of_beams,
    # Multiplier to penalize repeated n-grams
    # Higher values discourage repetition in the generated text
    "REPETITION_PENALTY": 3.0,
    # Penalty applied for producing long sequences
    # Higher values encourage longer sequences
    "LENGTH_PENALTY": 1.0,
    # The number of steps to take before the gradient is averaged and applied
    # Helps in stabilizing training and requires less memory
    "GRADIENT_ACCUMULATION_STEPS": 1,
    # Weight decay introduced to the optimizer to prevent over-fitting
    # Regularization strategy by adding a small penalty, typically the L2 norm of the weights
    "WEIGHT_DECAY": 0.0,
    # Small constant to prevent any division by zero in the implementation (Adam)
    "ADAM_EPSILON": 1e-8,
    # Number of steps for the warmup phase
    # Helps in avoiding very high and undesirable values of gradients at the start of training
    "WARMUP_STEPS": 2,
}

model_output_directory = "../models/" + input(
    "What name would you like to give the fine-tuned model?"
)

# Load JSONLdata
data = load_jsonl_data(
    "../data/training/training_validation_set.jsonl",
    BULLET_PROMPT_PREFIX,
    isDataFrame=True,
)

# Set device to be used based on GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logger.info(f"Loading {model_params['MODEL']}...")
# Model is sent to device (GPU/TPU) for using the hardware
model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
model = model.to(device)
# Tokenzier for encoding the text
tokenizer = T5Tokenizer.from_pretrained(
    model_params["MODEL"], model_max_length=model_params["MAX_SOURCE_TEXT_LENGTH"]
)

# Creating a custom dataset for reading the dataset and loading it into the dataloader
# to pass it to the neural network for fine tuning the model
class CustomDataset(Dataset):
    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        return len(self.target_text)

    def __getitem__(self, index):
        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # Cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }


# Generates a penalty for not complying to bullet formatting
def format_penalty(outputs, tokenizer, format_pattern):
    total_penalty = 0.0
    logits = outputs.logits
    # Converting the logits to token ids
    token_ids = torch.argmax(logits, dim=-1)
    # Decoding the token ids to text
    decoded_outputs = [
        tokenizer.decode(token_ids[i], skip_special_tokens=True)
        for i in range(token_ids.shape[0])
    ]

    for text in decoded_outputs:
        match = re.fullmatch(format_pattern, text)
        # If the output does not match the desired format exactly, add a penalty
        if not match:
            total_penalty += 1.0

    return torch.tensor(total_penalty, device=logits.device)


# Function to be called for training with the parameters passed from main function
def train(epoch, tokenizer, model, device, loader, optimizer, scheduler):
    model.train()
    for step, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        # Add a penalty to the loss for outputs that don't match the format
        format_loss = format_penalty(outputs, tokenizer, BULLET_PATTERN)
        total_loss = loss + format_loss

        if step % 10 == 0:
            training_logger.add_row(str(epoch + 1), str(step), str(total_loss))
            general_logger.print(training_logger)

        total_loss.backward()

        # Perform a backward pass (back propagation)
        if (step + 1) % model_params["GRADIENT_ACCUMULATION_STEPS"] == 0:
            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # Update parameters
            optimizer.step()
            # Clear gradients
            optimizer.zero_grad()
            # Adjust the learning rate based on the number of iterations
            scheduler.step()
        pass


# Function to evaluate model for predictions and compute ROUGE scores
def validate(tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []

    # Initialize the rouge scorer
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = []

    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data["target_ids"].to(device, dtype=torch.long)
            ids = data["source_ids"].to(device, dtype=torch.long)
            mask = data["source_mask"].to(device, dtype=torch.long)

            generated_ids = model.generate(
                input_ids=ids,
                attention_mask=mask,
                max_length=model_params["MAX_SOURCE_TEXT_LENGTH"],
                num_beams=model_params["NUM_BEAMS"],
                repetition_penalty=model_params["REPETITION_PENALTY"],
                length_penalty=model_params["LENGTH_PENALTY"],
                early_stopping=True,
            )
            preds = [
                tokenizer.decode(
                    g, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
                for g in generated_ids
            ]
            targets = [
                tokenizer.decode(
                    t, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
                for t in y
            ]

            # Calculate rouge scores for each prediction and corresponding target
            for pred, target in zip(preds, targets):
                score = scorer.score(target, pred)
                scores.append(score)

            if _ % 10 == 0:
                general_logger.print(f"Completed {_}")

            predictions.extend(preds)
            actuals.extend(targets)

    # Compute the average ROUGE scores for the entire validation set
    avg_scores = {
        "rouge1": np.mean([score["rouge1"].fmeasure for score in scores]),
        "rouge2": np.mean([score["rouge2"].fmeasure for score in scores]),
        "rougeL": np.mean([score["rougeL"].fmeasure for score in scores]),
    }

    logger.info(f"Average ROUGE scores: {avg_scores}")

    return predictions, actuals


# T5 training main function
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir):
    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])
    np.random.seed(model_params["SEED"])
    torch.backends.cudnn.deterministic = True

    logger.info(f"Reading data...")
    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]

    # Creation of Dataset and Dataloader
    # 80% of the data will be used for training and the rest for validation
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    logger.info(f"FULL Dataset: {dataframe.shape}")
    logger.info(f"TRAIN Dataset: {train_dataset.shape}")
    logger.info(f"VALIDATION Dataset: {val_dataset.shape}")

    # Creating the Training and Validation dataset for further creation of data loader
    training_set = CustomDataset(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = CustomDataset(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of data loaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }
    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of data loaders for testing and validation - this will be used down for training and validation stage for the model
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session
    optimizer = torch.optim.AdamW(
        params=[p for p in model.parameters() if p.requires_grad],
        lr=model_params["LEARNING_RATE"],
        eps=model_params["ADAM_EPSILON"],
        weight_decay=model_params["WEIGHT_DECAY"],
    )

    # Define the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=model_params["WARMUP_STEPS"],
        num_training_steps=model_params["TRAIN_EPOCHS"]
        * len(training_loader)
        // model_params["GRADIENT_ACCUMULATION_STEPS"],
    )

    # Training loop
    logger.info(f"Initiating fine tuning...")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer, scheduler)
        pass

    logger.info(f"Saving model...")
    # Saving the model after training
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Evaluating validation dataset
    logger.info(f"Initiating validation...")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    general_logger.save_text(os.path.join(output_dir, "logs.txt"))

    logger.success(f"Model validation completed!")
    logger.info(f"Fine-tuned model saved to: {output_dir}")
    logger.info(
        f"Validation data saved to: {os.path.join(output_dir,'predictions.csv')}"
    )
    logger.info(f"Notebook logs saved to: {os.path.join(output_dir,'logs.txt')}")

# In case of interrupt, save model
def save_and_exit(signal, frame):
    logger.info("Saving model...")
    model.save_pretrained(model_output_directory)
    tokenizer.save_pretrained(model_output_directory)
    logger.success("Model saved. Shutting down...")
    sys.exit(0)  # Exit the process cleanly

# Attach the SIGINT signal (generated by Ctrl+C) to the handler
signal.signal(signal.SIGINT, save_and_exit)

# Run training function on the T5 model using data set and training parameters
T5Trainer(
    dataframe=data,
    source_text="input",
    target_text="output",
    model_params=model_params,
    output_dir=f"{model_output_directory}",
)

2023-07-15 23:30:11.256 | INFO     | __main__:T5Trainer:275 - Loading t5-large...
2023-07-15 23:30:22.920 | INFO     | __main__:T5Trainer:286 - Reading data...
2023-07-15 23:30:22.923 | INFO     | __main__:T5Trainer:298 - FULL Dataset: (450, 2)
2023-07-15 23:30:22.924 | INFO     | __main__:T5Trainer:299 - TRAIN Dataset: (360, 2)
2023-07-15 23:30:22.925 | INFO     | __main__:T5Trainer:300 - VALIDATION Dataset: (90, 2)
2023-07-15 23:30:22.930 | INFO     | __main__:T5Trainer:354 - Initiating fine tuning...


                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
+-------------------------------------------------------+

                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(17.4916, grad_fn=<AddBackward0>)|
+-------------------------------------------------------+

                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(17.4916, grad_fn=<AddBackward0>)|
|  2   |   0   | tensor(11.9839, grad_fn=<AddBackward0>)|
+-------------------------------------------------------+

                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(17.4916, grad_fn=<AddBackward0>)|
|  2   |   0   | tensor(11.9839, grad_fn=<AddBackward0>)|
|  2   |  10   | tensor(8.9112, grad_fn=<AddBackward0>) |
+-------------------------------------------------------+

...

                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(17.4916, grad_fn=<AddBackward0>)|
|  2   |   0   | tensor(11.9839, grad_fn=<AddBackward0>)|
|  2   |  10   | tensor(8.9112, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(4.4533, grad_fn=<AddBackward0>) |
|  3   |  10   | tensor(2.5062, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(2.2845, grad_fn=<AddBackward0>) |
+-------------------------------------------------------+

                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(17.4916, grad_fn=<AddBackward0>)|
|  2   |   0   | tensor(11.9839, grad_fn=<AddBackward0>)|
|  2   |  10   | tensor(8.9112, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(4.4533, grad_fn=<AddBackward0>) |
|  3   |  10   | tensor(2.5062, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(2.2845, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(3.1788, grad_fn=<AddBackward0>) |
+-------------------------------------------------------+

                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(17.4916, grad_fn=<AddBackward0>)|
|  2   |   0   | tensor(11.9839, grad_fn=<AddBackward0>)|
|  2   |  10   | tensor(8.9112, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(4.4533, grad_fn=<AddBackward0>) |
|  3   |  10   | tensor(2.5062, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(2.2845, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(3.1788, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(1.1532, grad_fn=<AddBackward0>) |
+-------------------------------------------------------+

                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(17.4916, grad_fn=<AddBackward0>)|
|  2   |   0   | tensor(11.9839, grad_fn=<AddBackward0>)|
|  2   |  10   | tensor(8.9112, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(4.4533, grad_fn=<AddBackward0>) |
|  3   |  10   | tensor(2.5062, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(2.2845, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(3.1788, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(1.1532, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(3.1195, grad_fn=<AddBackward0>) |
+-------------------------------------------------------+

                     Training Status                     
+-------------------------------------------------------+
|Epoch | Steps |                  Loss                  |
|------+-------+----------------------------------------|
|  1   |   0   | tensor(32.1623, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(6.0856, grad_fn=<AddBackward0>) |
|  1   |  20   | tensor(5.5537, grad_fn=<AddBackward0>) |
|  2   |   0   | tensor(7.5343, grad_fn=<AddBackward0>) |
|  2   |  10   | tensor(6.2988, grad_fn=<AddBackward0>) |
|  2   |  20   | tensor(6.0131, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(13.7477, grad_fn=<AddBackward0>)|
|  3   |  10   | tensor(10.7223, grad_fn=<AddBackward0>)|
|  3   |  20   | tensor(8.3484, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(8.9654, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(7.7426, grad_fn=<AddBackward0>) |
|  4   |  20   | tensor(4.7016, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(5.5619, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(5.6811, grad_fn=<AddBackward0>) |
|  5   |  20   | tensor(12.4023, grad_fn=<AddBackward0>)|
|  6   |   0   | tensor(6.8759, grad_fn=<AddBackward0>) |
|  6   |  10   | tensor(12.1846, grad_fn=<AddBackward0>)|
|  6   |  20   | tensor(10.0798, grad_fn=<AddBackward0>)|
|  7   |   0   | tensor(11.6262, grad_fn=<AddBackward0>)|
|  7   |  10   | tensor(9.7450, grad_fn=<AddBackward0>) |
|  7   |  20   | tensor(7.6934, grad_fn=<AddBackward0>) |
|  8   |   0   | tensor(6.1358, grad_fn=<AddBackward0>) |
|  8   |  10   | tensor(7.5864, grad_fn=<AddBackward0>) |
|  8   |  20   | tensor(2.1700, grad_fn=<AddBackward0>) |
|  9   |   0   | tensor(8.1567, grad_fn=<AddBackward0>) |
|  9   |  10   | tensor(3.2303, grad_fn=<AddBackward0>) |
|  9   |  20   | tensor(5.9111, grad_fn=<AddBackward0>) |
| 10   |   0   | tensor(9.9356, grad_fn=<AddBackward0>) |
| 10   |  10   | tensor(5.1729, grad_fn=<AddBackward0>) |
| 10   |  20   | tensor(6.7947, grad_fn=<AddBackward0>) |
| 11   |   0   | tensor(7.9997, grad_fn=<AddBackward0>) |
| 11   |  10   | tensor(3.6585, grad_fn=<AddBackward0>) |
| 11   |  20   | tensor(5.1835, grad_fn=<AddBackward0>) |
| 12   |   0   | tensor(3.5284, grad_fn=<AddBackward0>) |
| 12   |  10   | tensor(6.6684, grad_fn=<AddBackward0>) |
| 12   |  20   | tensor(5.5951, grad_fn=<AddBackward0>) |
| 13   |   0   | tensor(6.6966, grad_fn=<AddBackward0>) |
| 13   |  10   | tensor(2.5137, grad_fn=<AddBackward0>) |
| 13   |  20   | tensor(7.5238, grad_fn=<AddBackward0>) |
| 14   |   0   | tensor(3.4581, grad_fn=<AddBackward0>) |
| 14   |  10   | tensor(9.3945, grad_fn=<AddBackward0>) |
| 14   |  20   | tensor(7.4712, grad_fn=<AddBackward0>) |
| 15   |   0   | tensor(7.3581, grad_fn=<AddBackward0>) |
| 15   |  10   | tensor(2.2439, grad_fn=<AddBackward0>) |
| 15   |  20   | tensor(3.2466, grad_fn=<AddBackward0>) |
| 16   |   0   | tensor(3.2528, grad_fn=<AddBackward0>) |
| 16   |  10   | tensor(1.0546, grad_fn=<AddBackward0>) |
| 16   |  20   | tensor(6.2260, grad_fn=<AddBackward0>) |
|  1   |   0   | tensor(19.9717, grad_fn=<AddBackward0>)|
|  1   |  10   | tensor(17.4916, grad_fn=<AddBackward0>)|
|  2   |   0   | tensor(11.9839, grad_fn=<AddBackward0>)|
|  2   |  10   | tensor(8.9112, grad_fn=<AddBackward0>) |
|  3   |   0   | tensor(4.4533, grad_fn=<AddBackward0>) |
|  3   |  10   | tensor(2.5062, grad_fn=<AddBackward0>) |
|  4   |   0   | tensor(2.2845, grad_fn=<AddBackward0>) |
|  4   |  10   | tensor(3.1788, grad_fn=<AddBackward0>) |
|  5   |   0   | tensor(1.1532, grad_fn=<AddBackward0>) |
|  5   |  10   | tensor(3.1195, grad_fn=<AddBackward0>) |
|  6   |   0   | tensor(1.0758, grad_fn=<AddBackward0>) |
+-------------------------------------------------------+

## Step 3: Fine Tuned Model Manual Testing

In [3]:
# Manual Test Scripts
from transformers import T5ForConditionalGeneration, T5Tokenizer

from scripts.files import load_jsonl_data
from scripts.bullet_patterns import BULLET_PROMPT_PREFIX

input_model = input(
    "What is the target model's relative directory path or checkpoint name on Hugging Face?"
)

# Model generation parameter control object
model_params = {
    # Name of the pre-trained model that will be fine-tuned
    "MODEL": f"{input_model}",
    # Maximum number of tokens from source text that model accepts
    "MAX_SOURCE_TEXT_LENGTH": max_input_token_length,
    # Maximum number of tokens from target text that model generates
    "MAX_TARGET_TEXT_LENGTH": max_output_token_length,
    # Number of alternative sequences generated at each step
    # More beams improve results, but increase computation
    "NUM_BEAMS": number_of_beams,
    # Scales logits before soft-max to control randomness
    # Lower values (~0) make output more deterministic
    "TEMPERATURE": 0.7,
    # Limits generated tokens to top K probabilities
    # Reduces chances of rare word predictions
    "TOP_K": 50,
    # Applies nucleus sampling, limiting token selection to a cumulative probability
    # Creates a balance between randomness and determinism
    "TOP_P": 0.95,
}


# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(
    model_params["MODEL"], model_max_length=model_params["MAX_SOURCE_TEXT_LENGTH"]
)

# Load the data from the manual test file
data = load_jsonl_data(
    "../data/training/manual_test_set.jsonl", BULLET_PROMPT_PREFIX, isDataFrame=False
)
for line in data:
    # Preprocess input
    input_text = line["input"]
    expected_summary = line["output"]

    inputs = tokenizer.encode_plus(
        input_text, return_tensors="pt", truncation=True, max_length=512
    )

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=model_params["MAX_TARGET_TEXT_LENGTH"],
        num_beams=model_params["NUM_BEAMS"],
        temperature=model_params["TEMPERATURE"],
        top_k=model_params["TOP_K"],
        top_p=model_params["TOP_P"],
        early_stopping=True,
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Print results
    print(f"> INPUT TEXT: {input_text}")
    print(f"\t> EXPECTED SUMMARY: {expected_summary}")
    print(f"\t> GENERATED SUMMARY: {summary}\n")

> INPUT TEXT: During a structure fire incident, I deployed a handline and effectively knocked down and extinguished a kitchen blaze. Through my prompt response and actions, I saved the dining facility, preventing an estimated $3.4 million in damages.
	> EXPECTED SUMMARY: - Deployed handline into structure fire; knocked down & extinguished kitchen blaze--saved $3.4M dining facility
	> GENERATE SUMMARY: - Structured building fire incident; deployed handlined kitchen blazed kitchen blazed kitchen blazed kitchen blazed kitchen blazed kitchen blaze

> INPUT TEXT: As a team leader, I led a three-member team in addressing multiple engine discrepancies on E-8C aircraft. We successfully replaced 12 intake/exhaust rivets, resulting in a cost savings of $2.4 million by avoiding the need for complete engine replacements.
	> EXPECTED SUMMARY: - Led 3-member team on multiple E-8C engine discrepancies; replaced 12 intake/exhaust rivets--saved $2.4M replacement cost
	> GENERATE SUMMARY: - Team leader;